# arXiv:1804.03680 [Hierarchical quantum classifiers](https://arxiv.org/abs/1804.03680)

2.5 Experimental results: Iris dataset

In [ ]:
from __future__ import annotations

import sys
import math
from collections.abc import Callable, Sequence
import networkx as nx
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import cupy as cp
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qmlant.models.binary_classification import TTN
from qmlant.neural_networks import (
    EstimatorTN,
    circuit_to_einsum_expectation,
)
import qmlant.optim as optim
from qmlant.visualization import draw_quimb_tn

from qmlant.datasets import Iris
from qmlant.transforms import ToTensor, MapLabel

## Data preparation for Versicolor/Versinica

In [ ]:
target_transform = transforms.Compose([
    MapLabel([1, 2], [1, -1]),
    ToTensor(int)
])

trainset = Iris(
    test_size=0.3,
    transform=ToTensor(),
    target_transform=target_transform,
    subclass_targets=[1, 2]
)

testset = Iris(
    train=False,
    test_size=0.3,
    transform=ToTensor(),
    target_transform=target_transform,
    subclass_targets=[1, 2]
)

## Quantum circuit

In [ ]:
n_qubits = 4

# Eqn. (1)
def make_init_circuit(
    n_qubits: int,
    dry_run: bool = False
) -> QuantumCircuit | int:
    if dry_run:
        return n_qubits

    init_circuit = QuantumCircuit(n_qubits)
    x = ParameterVector("x", n_qubits)
    for i in range(n_qubits):
        init_circuit.ry(x[i], i)

    return init_circuit

# Fig. 1 (a) TTN classifier
def make_ansatz(
    n_qubits: int,
    insert_barrier: bool = False,
    dry_run: bool = False
) -> QuantumCircuit | int:
    def append_U(qc, i, j, thetas, count, last_unitary=False, reverse=False):
        qc.ry(thetas[count], i)
        count += 1
        qc.ry(thetas[count], j)
        count += 1

        if reverse:
            ansatz.cx(j, i)
        else:
            ansatz.cx(i, j)
        if last_unitary:
            qc.ry(thetas[count], j)
            count += 1
        return count

    length = 2*n_qubits//2  # U5 - U6
    length += 3*n_qubits//4  # U7

    if dry_run:
        return length

    thetas = ParameterVector('θ', length)

    count = 0
    ansatz = QuantumCircuit(n_qubits)
    # U5 - U6
    reverse = False
    for i in range(0, n_qubits, 2):
        if i+1 >= n_qubits:
            break
        count = append_U(ansatz, i, i+1, thetas, count, reverse=reverse)
        reverse = not reverse
    if insert_barrier:
        ansatz.barrier()
    # U7
    for i in range(1, n_qubits, 4):
        if i+1 >= n_qubits:
            break
        count = append_U(ansatz, i, i+1, thetas, count, last_unitary=True)
    if insert_barrier:
        ansatz.barrier()
    assert count == length, count
    return ansatz

def make_placeholder_circuit(
    n_qubits: int,
    insert_barrier: bool = False,
    dry_run: bool = False
) -> QuantumCircuit | int:
    if dry_run:
        length_feature = make_init_circuit(n_qubits, dry_run=True)
        length_ansatz = make_ansatz(n_qubits, dry_run=True)
        length = length_feature + length_ansatz
        return length

    qc = make_init_circuit(n_qubits)
    ansatz = make_ansatz(n_qubits, insert_barrier)
    qc.compose(ansatz, inplace=True)

    return qc

placeholder_circuit = make_placeholder_circuit(n_qubits)
display(placeholder_circuit.draw())

## Define the Hamiltonian

In [ ]:
hamiltonian = "IIZI"  # 3rd position from the left, c.f. Fig. 1

## Check locations of parameters in the TensorNetwork

In [ ]:
_, _, name2locs = circuit_to_einsum_expectation(placeholder_circuit, hamiltonian)

print(name2locs)

## Show TensorNetwork structure

In [ ]:
draw_quimb_tn(placeholder_circuit, hamiltonian, True)

## Train the circuit

If scipy-based optimization would be preffered, the Appendix is available.

In [ ]:
class PQCTrainerTN:
    def __init__(self,
        qc: QuantumCircuit,
        initial_point: Sequence[float],
        optimizer: optim.Optimizer
    ):
        self.qc_pl = qc  # placeholder circuit
        self.initial_point = np.array(initial_point)
        self.optimizer = optimizer

    def fit(self,
        dataset: Dataset,
        batch_size: int,
        operator: str,
        callbacks: list[Callable] | None = None,
        epochs: int = 1
    ) -> None:
        expr, operands, pname2locs = circuit_to_einsum_expectation(self.qc_pl, operator)

        dataloader = DataLoader(dataset, batch_size, shuffle=True, drop_last=True)
        callbacks = callbacks if callbacks is not None else []

        opt_loss = sys.maxsize
        opt_params = None
        params = self.initial_point.copy()
        if isinstance(params, list):
            params = np.array(params)

        qnn = EstimatorTN(pname2locs, expr, operands)

        for epoch in range(epochs):
            for batch, label in dataloader:
                batch, label = self._preprocess_batch(batch, label)
                label = label.reshape(label.shape[0], -1)

                # "forward"
                expvals = qnn.forward(params, batch)
                total_loss = np.mean((expvals - label)**2)

                # "backward"
                # The parameter-shift rule
                # [[∂f1/∂θ1, ∂f1/∂θ2, ..., ∂f1/∂θn],
                #  [∂f2/∂θ1, ∂f2/∂θ2, ..., ∂f2/∂θn],
                #  ...]
                grads = qnn.backward()
                expvals_minus_label = (expvals - label).reshape(batch.shape[0], -1)
                total_grads = np.mean(expvals_minus_label * grads, axis=0)

                for callback in callbacks:
                    callback(total_loss, params)

                # "update params"
                self.optimizer.update(params, total_grads)

    def _preprocess_batch(self,
        batch: torch.Tensor,
        label: torch.Tensor
    ) -> tuple[np.ndarray, np.ndarray]:
        batch = batch.detach().numpy()
        label = label.detach().numpy()
        return batch, label

In [ ]:
def RunPQCTrain(
    dataset: Dataset,
    batch_size: int,
    qc: QuantumCircuit,
    operator: str,
    init: Sequence[float] | None = None,
    epochs: int = 1,
    interval: int = 100
):
    opt_params = None
    opt_loss = None

    def save_opt_params(loss, params):
        nonlocal opt_params, opt_loss

        if opt_loss is None or loss < opt_loss:
            opt_params = params.copy()
            opt_loss = loss

    # Store intermediate results
    history = {"loss": [], "params": []}
    cnt = -1

    def store_intermediate_result(loss, params):
        nonlocal cnt

        history["loss"].append(loss)
        history["params"].append(None)
        cnt += 1
        if cnt % interval != 0:
            return
        print(f'{loss=}')

    optimizer = optim.Adam(alpha=0.5)
    trainer = PQCTrainerTN(qc=qc, initial_point=init, optimizer=optimizer)
    trainer.fit(dataset, batch_size, operator,
                callbacks=[save_opt_params, store_intermediate_result],
                epochs=epochs)

    return opt_params, history["loss"]

In [ ]:
%%time

length = make_ansatz(n_qubits, dry_run=True)
placeholder_circuit = make_placeholder_circuit(n_qubits)

rng = np.random.default_rng(10)
init = rng.random(length) * 2*math.pi

opt_params, loss_list = RunPQCTrain(trainset, 64,
                                    placeholder_circuit, hamiltonian, init=init,
                                    epochs=15, interval=10)

print(f'final loss={loss_list[-1]}')
print(f'{opt_params=}')

## Validate results

### Measure test acc

In [ ]:
testloader = DataLoader(testset, 32)

qc_pl = make_placeholder_circuit(n_qubits)
expr, operands, pname2locs = circuit_to_einsum_expectation(qc_pl, hamiltonian)

qnn = EstimatorTN(pname2locs, expr, operands)

total = 0
total_correct = 0

for i, (batch, label) in enumerate(testloader):
    batch, label = batch.detach().numpy(), label.detach().numpy()
    label = label.reshape(label.shape[0], -1)

    # "forward"
    expvals = qnn.forward(opt_params, batch)

    predict_labels = np.ones_like(expvals)
    predict_labels[np.where(expvals < 0)] = -1
    predict_labels = predict_labels.astype(int)

    total_correct += np.sum(predict_labels == label)
    total += batch.shape[0]

print(f'test acc={np.round(total_correct/total, 2)}')

### Visualize loss_list

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(loss_list)), loss_list)
plt.show()

# Appendix

## Using SciPy optimizers

In [ ]:
from scipy.optimize import minimize

### COBYLA

In [ ]:
class PQCTrainerTN_COBYLA:
    def __init__(self,
        qc: QuantumCircuit,
        initial_point: Sequence[float]
    ):
        self.qc_pl = qc  # placeholder circuit
        self.initial_point = np.array(initial_point)

    def fit(self,
        dataset: Dataset,
        operator: str,
        callbacks: list[Callable] | None = None,
        epochs: int = 1
    ) -> None:
        expr, operands, pname2locs = circuit_to_einsum_expectation(self.qc_pl, operator)

        # full batch
        dataloader = DataLoader(dataset, len(dataset), shuffle=True, drop_last=True)
        callbacks = callbacks if callbacks is not None else []

        opt_loss = sys.maxsize
        opt_params = None
        params = self.initial_point.copy()
        if isinstance(params, list):
            params = np.array(params)

        qnn = EstimatorTN(pname2locs, expr, operands)

        batch, label = next(iter(dataloader))
        batch, label = self._preprocess_batch(batch, label)
        label = label.reshape(label.shape[0], -1)

        loss_list = []

        def cost(x, *args) -> float:
            nonlocal loss_list, callbacks
            
            params = x
            qnn, batch, label = args
            expvals = qnn.forward(params, batch)
            loss = np.mean((expvals - label)**2)

            for callback in callbacks:
                callback(loss, params)

            loss_list.append(loss)
            return loss

        result = minimize(
            cost,
            params,
            args=(qnn, batch, label),
            method="COBYLA",
            options={
                "maxiter": epochs
            },
        )
        return result, loss_list

    def _preprocess_batch(self,
        batch: torch.Tensor,
        label: torch.Tensor
    ) -> tuple[np.ndarray, np.ndarray]:
        batch = batch.detach().numpy()
        label = label.detach().numpy()
        return batch, label

In [ ]:
def RunPQCTrain_COBYLA(
    dataset: Dataset,
    qc: QuantumCircuit,
    operator: str,
    init: Sequence[float] | None = None,
    epochs: int = 1,
    interval: int = 100
):
    opt_params = None
    opt_loss = None

    def save_opt_params(loss, params):
        nonlocal opt_params, opt_loss

        if opt_loss is None or loss < opt_loss:
            opt_params = params.copy()
            opt_loss = loss

    # Store intermediate results
    history = {"loss": [], "params": []}
    cnt = -1

    def store_intermediate_result(loss, params):
        nonlocal cnt

        history["loss"].append(loss)
        history["params"].append(None)
        cnt += 1
        if cnt % interval != 0:
            return
        print(f'{loss=}')
    
    trainer = PQCTrainerTN_COBYLA(qc=qc, initial_point=init)
    result, loss_list = trainer.fit(
        dataset,
        operator,
        callbacks=[save_opt_params, store_intermediate_result],
        epochs=epochs
    )

    return result["x"], loss_list

In [ ]:
%%time

length = make_ansatz(n_qubits, dry_run=True)
placeholder_circuit = make_placeholder_circuit(n_qubits)

rng = np.random.default_rng(10)
init = rng.random(length) * 2*math.pi

opt_params, loss_list = RunPQCTrain_COBYLA(
    trainset, placeholder_circuit, hamiltonian, init=init, epochs=25, interval=10
)

print(f'final loss={loss_list[-1]}')
print(f'{opt_params=}')

In [ ]:
testloader = DataLoader(testset, 32)

qc_pl = make_placeholder_circuit(n_qubits)
expr, operands, pname2locs = circuit_to_einsum_expectation(qc_pl, hamiltonian)

qnn = EstimatorTN(pname2locs, expr, operands)

total = 0
total_correct = 0

for i, (batch, label) in enumerate(testloader):
    batch, label = batch.detach().numpy(), label.detach().numpy()
    label = label.reshape(label.shape[0], -1)

    # "forward"
    expvals = qnn.forward(opt_params, batch)

    predict_labels = np.ones_like(expvals)
    predict_labels[np.where(expvals < 0)] = -1
    predict_labels = predict_labels.astype(int)

    total_correct += np.sum(predict_labels == label)
    total += batch.shape[0]

print(f'test acc={np.round(total_correct/total, 2)}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(loss_list)), loss_list)
plt.show()